In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from string import punctuation
from random import sample
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import nltk
nltk.download('wordnet')
from sklearn.model_selection import GridSearchCV


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sawag\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:

df1 = pd.read_csv("Data/Eclipse/EP_dup.csv", sep=";")
df2 = pd.read_csv("Data/Eclipse/EP_nondup.csv", sep=";")
df3 = pd.read_csv("Data/Mozilla/M_Duplicate BRs.csv", sep=";")  
df4 = pd.read_csv("Data/Mozilla/M_NonDuplicate BRs.csv", sep=";")
df5 = pd.read_csv("Data/ThunderBird/dup_TB.csv", sep=";")
df6 = pd.read_csv("Data/ThunderBird/Nondup_TB.csv", sep=";")

frames=[df1,df2,df3,df4,df5,df6]
data=pd.concat(frames)
data

,Issue_id,Duplicated_issue,Title1,Description1,Title2,Description2,Label
0,25,28126,cvs ui need vcm prefs default repo connection gc,it would be helpful if there was a notion of d...,wizards patch standard public cvs repositories,this patch adds a convenient way to check thin...,1
1,40,20,need connect to team stream gcqpkw,i would like to be able to connect to a team s...,workspace files,thought it would be useful if the set of repo ...,1
2,48,22,make sure can future store other project refer...,project references come in three flavours . p...,persist sharing recommendations and project ve...,project descriptions dont store sharing recomm...,1
3,61,60,.vcmmeta showing as change gdqtgw,useruser install drop into declipse user ...,need custom .vcmignore comparemerge gdqt,useruser install drop into declipse user ...,1
4,94,2,repositories view all file types open to the t...,when browsing files in the repositories view i...,opening repository resources doesnt honor type...,opening repository resource open the default ...,1
...,...,...,...,...,...,...,...
9900,418747,953052,thunderbird outlook migrate lost outlooks pst ...,i installed thunderbird hoping i could impor...,i am using thunderbird esr latest but i keep ...,i am using thunderbird .. esr latest but i...,0
9901,418821,953165,urls in mail with comma opens with c in browser,when clicking in text mail not html on url w...,picture in feed not showing,. i linked the rssfeed from httpwww.nu.nlf...,0
9902,418832,953249,greek attachments not displayed correctly,i am using gmail imap accounts. when i recei...,add the flagcolpng to linux,httpmxr.mozilla.orgcommcentralsearchstringflag...,0
9903,419035,953375,branch only random crash when i want to displ...,the problem appears with icedove the debian ...,replace undefined gpromptservice with services...,patch v discovered a call to undefined gprom...,0


In [3]:
data = data.reset_index()

In [4]:
data['TitleDescription1'] = data['Title1'].str.cat(data['Description1'],sep=" ")
data['TitleDescription2'] = data['Title2'].str.cat(data['Description2'],sep=" ")

In [5]:
stop_words = set(stopwords.words('english'))

In [6]:
def words(text, remove_stop_words=True, stem_words=False):
    # Remove punctuation from questions
    text = ''.join([c for c in text if c not in punctuation])
    
    # Lowering the words in questions
    text = text.lower()
    lemmatizer=WordNetLemmatizer()
    stemmer=PorterStemmer()

    # Remove stop words from questions
    if remove_stop_words:
        text = text.split()
    #    text = [w for w in text if not w in stop_words]
     #   text = " ".join(text)
        text = [lemmatizer.lemmatize(word.lower()) for word in text if not word in stop_words]
        text=' '.join(text)
        #text=[stemmer.stem(word) for word in text if word not in stop_words]
        #text=' '.join(text)
    # Return a list of words
    return(text)

In [7]:
def process(bug_list, bugs):
    for bug in bugs:
        bug_list.append(words(bug))
processed_bug1 = []
processed_bug2 = []
process(processed_bug1, data.TitleDescription1)
process(processed_bug2, data.TitleDescription2)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer = 'word',
                        stop_words = 'english',
                        lowercase = True,
                        max_features = 300,
                        norm = 'l1')

In [9]:
words = pd.concat([data.TitleDescription1, data.TitleDescription2], axis = 0)

In [10]:
tfidf.fit(words)
duplicate_1 = tfidf.transform(data.TitleDescription1)
duplicate_2 = tfidf.transform(data.TitleDescription2)

In [11]:
x = abs(duplicate_1 - duplicate_2)
y = data['Label']

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Conv1D, GlobalMaxPooling1D


model = Sequential()
model.add(Dense(64,input_shape=(300,)))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                19264     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
activation (Activation)      (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6

In [14]:
results = model.fit(x_train.todense(),y_train,batch_size=64,epochs=5,verbose=1,validation_data=(x_test.todense(),y_test))

Epoch 1/5
1526/1526 [==============================] - 2s 1ms/step - loss: 0.4223 - acc: 0.8187 - val_loss: 0.3441 - val_acc: 0.8644
Epoch 2/5
1526/1526 [==============================] - 2s 1ms/step - loss: 0.3472 - acc: 0.8626 - val_loss: 0.3394 - val_acc: 0.8649
Epoch 3/5
1526/1526 [==============================] - 2s 1ms/step - loss: 0.3360 - acc: 0.8670 - val_loss: 0.3272 - val_acc: 0.8698
Epoch 4/5
1526/1526 [==============================] - 2s 1ms/step - loss: 0.3276 - acc: 0.8707 - val_loss: 0.3229 - val_acc: 0.8717
Epoch 5/5
1526/1526 [==============================] - 2s 1ms/step - loss: 0.3217 - acc: 0.8738 - val_loss: 0.3236 - val_acc: 0.8717


In [15]:
from sklearn.metrics import classification_report
pred = model.predict_classes(x_test)
print(classification_report(y_test,pred,labels=[1,0]))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
              precision    recall  f1-score   support

           1       0.88      0.72      0.79      8294
           0       0.87      0.95      0.91     16121

    accuracy                           0.87     24415
   macro avg       0.87      0.84      0.85     24415
weighted avg       0.87      0.87      0.87     24415



In [ ]:
#init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 
        #     'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']

In [ ]:
#param_grid = dict(init_mode=init_mode)

In [32]:
batch_size = [10,20,30,50,100]
epochs = [10, 50,100]
param_grid = dict(batch_size=batch_size, epochs=epochs)

In [33]:
def cm():
    
    model = Sequential()
    model.add(Dense(64,input_shape=(300,)))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(64))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])
    return model

In [34]:
from keras.wrappers.scikit_learn import KerasClassifier


In [35]:
model = KerasClassifier(build_fn=cm, verbose=0)


In [22]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
grid_result = grid.fit(x_train.todense(), y_train)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                19264     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
activation_3 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
activation_4 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 64)                19264     
_________________________________________________________________
dropout_14 (Dropout)         (None, 64)                0         
_________________________________________________________________
activation_21 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_22 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_15 (Dropout)         (None, 64)                0         
_________________________________________________________________
activation_22 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 1)                

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_39 (Dense)             (None, 64)                19264     
_________________________________________________________________
dropout_26 (Dropout)         (None, 64)                0         
_________________________________________________________________
activation_39 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_40 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_27 (Dropout)         (None, 64)                0         
_________________________________________________________________
activation_40 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_41 (Dense)             (None, 1)               

In [23]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.870213 using {'batch_size': 10, 'epochs': 10}


In [24]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.870213 (0.001298) with: {'batch_size': 10, 'epochs': 10}
0.861079 (0.001515) with: {'batch_size': 10, 'epochs': 50}
0.869445 (0.001647) with: {'batch_size': 20, 'epochs': 10}
0.862564 (0.000790) with: {'batch_size': 20, 'epochs': 50}


In [31]:
predictions = grid_result.predict(x_test)
print(classification_report(y_test, predictions,labels=[1,0]))

              precision    recall  f1-score   support

           1       0.88      0.73      0.80      8294
           0       0.87      0.95      0.91     16121

    accuracy                           0.87     24415
   macro avg       0.87      0.84      0.85     24415
weighted avg       0.87      0.87      0.87     24415



In [36]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
grid_result = grid.fit(x_train.todense(), y_train)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_42 (Dense)             (None, 64)                19264     
_________________________________________________________________
dropout_28 (Dropout)         (None, 64)                0         
_________________________________________________________________
activation_42 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_43 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_29 (Dropout)         (None, 64)                0         
_________________________________________________________________
activation_43 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_44 (Dense)             (None, 1)               

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_60 (Dense)             (None, 64)                19264     
_________________________________________________________________
dropout_40 (Dropout)         (None, 64)                0         
_________________________________________________________________
activation_60 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_61 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_41 (Dropout)         (None, 64)                0         
_________________________________________________________________
activation_61 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_62 (Dense)             (None, 1)               

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_78 (Dense)             (None, 64)                19264     
_________________________________________________________________
dropout_52 (Dropout)         (None, 64)                0         
_________________________________________________________________
activation_78 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_79 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_53 (Dropout)         (None, 64)                0         
_________________________________________________________________
activation_79 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_80 (Dense)             (None, 1)               

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_96 (Dense)             (None, 64)                19264     
_________________________________________________________________
dropout_64 (Dropout)         (None, 64)                0         
_________________________________________________________________
activation_96 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_97 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_65 (Dropout)         (None, 64)                0         
_________________________________________________________________
activation_97 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_98 (Dense)             (None, 1)               

Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_114 (Dense)            (None, 64)                19264     
_________________________________________________________________
dropout_76 (Dropout)         (None, 64)                0         
_________________________________________________________________
activation_114 (Activation)  (None, 64)                0         
_________________________________________________________________
dense_115 (Dense)            (None, 64)                4160      
_________________________________________________________________
dropout_77 (Dropout)         (None, 64)                0         
_________________________________________________________________
activation_115 (Activation)  (None, 64)                0         
_________________________________________________________________
dense_116 (Dense)            (None, 1)               

Model: "sequential_44"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_132 (Dense)            (None, 64)                19264     
_________________________________________________________________
dropout_88 (Dropout)         (None, 64)                0         
_________________________________________________________________
activation_132 (Activation)  (None, 64)                0         
_________________________________________________________________
dense_133 (Dense)            (None, 64)                4160      
_________________________________________________________________
dropout_89 (Dropout)         (None, 64)                0         
_________________________________________________________________
activation_133 (Activation)  (None, 64)                0         
_________________________________________________________________
dense_134 (Dense)            (None, 1)               

Model: "sequential_50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_150 (Dense)            (None, 64)                19264     
_________________________________________________________________
dropout_100 (Dropout)        (None, 64)                0         
_________________________________________________________________
activation_150 (Activation)  (None, 64)                0         
_________________________________________________________________
dense_151 (Dense)            (None, 64)                4160      
_________________________________________________________________
dropout_101 (Dropout)        (None, 64)                0         
_________________________________________________________________
activation_151 (Activation)  (None, 64)                0         
_________________________________________________________________
dense_152 (Dense)            (None, 1)               

Model: "sequential_56"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_168 (Dense)            (None, 64)                19264     
_________________________________________________________________
dropout_112 (Dropout)        (None, 64)                0         
_________________________________________________________________
activation_168 (Activation)  (None, 64)                0         
_________________________________________________________________
dense_169 (Dense)            (None, 64)                4160      
_________________________________________________________________
dropout_113 (Dropout)        (None, 64)                0         
_________________________________________________________________
activation_169 (Activation)  (None, 64)                0         
_________________________________________________________________
dense_170 (Dense)            (None, 1)               

In [37]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.870254 (0.001045) with: {'batch_size': 10, 'epochs': 10}
0.862922 (0.001002) with: {'batch_size': 10, 'epochs': 50}
0.857403 (0.003474) with: {'batch_size': 10, 'epochs': 100}
0.869281 (0.001139) with: {'batch_size': 20, 'epochs': 10}
0.862421 (0.002112) with: {'batch_size': 20, 'epochs': 50}
0.859144 (0.000646) with: {'batch_size': 20, 'epochs': 100}
0.869209 (0.000606) with: {'batch_size': 30, 'epochs': 10}
0.861960 (0.001760) with: {'batch_size': 30, 'epochs': 50}
0.858468 (0.001369) with: {'batch_size': 30, 'epochs': 100}
0.868953 (0.001873) with: {'batch_size': 50, 'epochs': 10}
0.862933 (0.001957) with: {'batch_size': 50, 'epochs': 50}
0.857854 (0.000866) with: {'batch_size': 50, 'epochs': 100}
0.870418 (0.001047) with: {'batch_size': 100, 'epochs': 10}
0.863137 (0.001508) with: {'batch_size': 100, 'epochs': 50}
0.856912 (0.001020) with: {'batch_size': 100, 'epochs': 100}


In [38]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.870418 using {'batch_size': 100, 'epochs': 10}


In [39]:
predictions = grid_result.predict(x_test)
print(classification_report(y_test, predictions,labels=[1,0]))

              precision    recall  f1-score   support

           1       0.87      0.74      0.80      8294
           0       0.87      0.94      0.91     16121

    accuracy                           0.87     24415
   macro avg       0.87      0.84      0.85     24415
weighted avg       0.87      0.87      0.87     24415

